In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('../..')

os.environ["OMP_NUM_THREADS"] = "4"

In [2]:
from pathlib import Path

from hydra import compose, initialize

import pandas as pd
import torch 
import numpy as np 

from src.models.recbole import RecboleBench
from src.preprocessing import ClassicDataset
from src.utils.logging import get_logger
from src.utils.processing import data_split
from src.utils.metrics import run_all_metrics, coverage

logger = get_logger(name=__name__)

%load_ext autoreload
%autoreload 2

/home/user/conda/envs/recsys/lib/python3.9/site-packages/numba/core/config.py:154: RuntimeWarning: environ NUMBA_NUM_THREADS defined but failed to parse ''
  warnings.warn("environ %s defined but failed to parse '%s'" %


In [3]:
# Import recbole Reqired Packages
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.utils import init_seed, init_logger
from recbole.quick_start import run_recbole

In [4]:
with initialize(config_path='../../config', version_base=None):
    cfg = compose(config_name='config')

In [5]:
"""RecBole module"""
from pathlib import Path
import os

from omegaconf import DictConfig

import omegaconf
import pandas as pd
import shutil

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.utils import init_seed

from src.models.recbole import RecboleBench
from src.preprocessing import ClassicDataset
from src.utils.logging import get_logger
from src.utils.processing import data_split, save_results
from src.utils.metrics import run_all_metrics, coverage


logger = get_logger(name=__name__)

In [6]:
recbole_name: str = cfg["library"]["name"]
cfg_data = cfg["dataset"]
cfg_model = cfg["library"]["recbole_model"]

dataset = ClassicDataset()
dataset.prepare(cfg_data)

data_split(
    dataset.prepared_data,
    cfg_data,
    cfg_model,
    return_format="recbole",
)

parameter_dict = omegaconf.OmegaConf.to_container(cfg_model["recbole_params"])
parameter_dict["data_path"] = os.path.join("data", "tmp")
parameter_dict["dataset"] = cfg_data["name"]


In [7]:

config = Config(
    model=cfg_model["name"],
    dataset=cfg_data["name"],
    config_file_list=None,
    config_dict=parameter_dict,
)

init_seed(config["seed"], config["reproducibility"])

dataset = create_dataset(config)

train_set, valid_set, test_set = data_preparation(config, dataset)

model_folder = Path(
    "/".join(
        ("preproc_data", cfg_data["name"], recbole_name, cfg_model["name"])
    )
)


In [8]:
# load trainval to make refit
config["benchmark_filename"] = ["trainval", "val", "test"]
dataset = create_dataset(config)
trainval_set, valid_set, test_set = data_preparation(config, dataset)


In [9]:
torch.cuda.set_device(1)
torch.cuda.empty_cache()

In [10]:

model = RecboleBench.initialize_with_params(
    train_loader=trainval_set, model_params={'latent_dimendion': 64, 'anneal_cap': 0.32176717760026463, 'dropout_prob': 0.618995872263191, 'learning_rate': 0.00030769747791639933} 
)
model.fit(trainval_set)
model.save_model(model_folder)


Max value of user's history interaction records has reached 32.41982842996143% of the total.
Train     0:   0%|                                                           | 0/98 [00:04<?, ?it/s]


AssertionError: Invalid device id

In [ ]:

ranks = model.get_relevant_ranks(test_set)
top_100_items = model.recommend_k(test_set, 100)

metrics = run_all_metrics(ranks, [5, 10, 20, 100])
coverage_metrics = []
for k in (5, 10, 20, 100):
    coverage_metrics.append(
        coverage(top_100_items, test_set.dataset.item_num, k)
    )

metrics_df = pd.DataFrame(
    metrics,
    index=[5, 10, 20, 100],
    columns=(
        "Precision@k",
        "Recall@K",
        "MAP@K",
        "nDCG@k",
        "MRR@k",
        "HitRate@k",
    ),
)
metrics_df["Coverage@K"] = coverage_metrics

metrics_df["Time_fit"] = bench.learning_time
metrics_df["Time_predict"] = bench.predict_time

save_results(
    (metrics_df, f"results_wasOptimized_{was_optimized}"),
    (top_100_items, f"items_wasOptimized_{was_optimized}"),
    (ranks, f"ranks_wasOptimized_{was_optimized}"),
    cfg["results_folder"],
    cfg_model["name"],
    cfg_data["name"],
    recbole_name,
)

# shutil.rmtree(os.path.join(parameter_dict["data_path"],cfg_data["name"]))

In [ ]:
recbole_name: str = cfg['library']['name']
cfg_data = cfg['dataset']
cfg_model = cfg['library']['recbole_model']

In [ ]:
ds = ClassicDataset()
ds.prepare(cfg_data)

In [ ]:
data_split(
            ds.prepared_data, 
            cfg_data, 
            cfg_model, 
            return_format='recbole',
        )  

In [ ]:
parameter_dict = {
'state': 'DEBUG',
'data_path': os.path.join('data', 'tmp'),
'dataset': cfg_data['name'],
'benchmark_filename': ['train', 'val', 'test'],
'load_col': {'inter': ['user_id', 'item_id', 'rating']},
'neg_sampling': None,
'train_neg_sample_args': None,
'worker': 8,
'gpu_id': '1,2',
'metrics': ['Precision', 'Recall', 'MAP', 'NDCG', 'MRR', 'HIT', 'ItemCoverage'],
'show_progress': True,
'eval_batch_size': 32768,
'valid_metric': 'NDCG@10'
        }

In [ ]:
config = Config(
        model="MultiVAE",
        dataset=cfg_data['name'],
        config_file_list=None,
        config_dict=parameter_dict,
            )

In [ ]:
config['nproc'] 

2

In [ ]:
init_seed(config["seed"], config["reproducibility"])

In [ ]:
dataset = create_dataset(config)


In [ ]:
train_set, valid_set, test_set = data_preparation(config, dataset)

In [ ]:
model_folder = Path('/'.join(
            ('preproc_data', cfg_data['name'], recbole_name, cfg_model['name'])
        ))

In [ ]:
model = RecboleBench.initialize_with_params(
                    train_loader=train_set
                )

Max value of user's history interaction records has reached 40.32697547683924% of the total.


In [ ]:
model.fit(train_set, valid_set)

Evaluate   : 100%|█████████████████████████| 10/10 [00:01<00:00,  8.04it/s, GPU RAM: 0.05 G/79.15 G]


In [ ]:
ranks = model.get_relevant_ranks(test_set)
top_100_items = model.recommend_k(test_set, 100)

/home/user/conda/envs/recsys/lib/python3.9/site-packages/recbole/utils/case_study.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  uid_series = torch.tensor(uid_series)


In [ ]:
metrics = run_all_metrics(ranks, [5, 10, 20, 100])
coverage_metrics = []
for k in (5, 10, 20, 100):
    coverage_metrics.append(coverage(
        top_100_items,
        test_set.dataset.item_num,
        k
    ))

metrics_df = pd.DataFrame(metrics, index=[5, 10, 20, 100], columns=(
    'Precision@k', 'Recall@K', 'MAP@K', 'nDCG@k', 'MRR@k', 'HitRate@k'
))
metrics_df['Coverage@K'] = coverage_metrics

metrics_df['Time_fit'] = model.learning_time
metrics_df['Time_predict'] = model.predict_time


In [ ]:
# Run recbole
results = run_recbole(model=config['model'], dataset=config['dataset'], config_dict=parameter_dict)

15 Sep 17:33    INFO  ['/home/user/conda/envs/recsys/lib/python3.9/site-packages/ipykernel_launcher.py', '--ip=127.0.0.1', '--stdin=9013', '--control=9011', '--hb=9010', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"81b872d8-3869-4564-b5ca-0d42a354f833"', '--shell=9012', '--transport="tcp"', '--iopub=9014', '--f=/home/jovyan/.local/share/jupyter/runtime/kernel-v2-1318495p11ieVVSukkU.json']
15 Sep 17:33    INFO  
General Hyper Parameters:
gpu_id = 1,2
use_gpu = True
seed = 2020
state = DEBUG
reproducibility = True
data_path = data/tmp/foursquare
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_n

### Compare recbole buildin training and evaluation and our pipeline

In [ ]:
metrics_df

,Precision@k,Recall@K,MAP@K,nDCG@k,MRR@k,HitRate@k,Coverage@K,Time_fit,Time_predict
5,0.148325,0.105326,0.083216,0.139763,0.240343,0.439206,0.133515,66.182008,0.101302
10,0.178451,0.168073,0.076406,0.151004,0.258215,0.575682,0.182561,66.182008,0.101302
20,0.295444,0.293852,0.089033,0.197353,0.271311,0.755583,0.258856,66.182008,0.101302
100,0.738223,0.738223,0.122651,0.333990,0.277047,0.956576,0.673025,66.182008,0.101302


In [ ]:
results['test_result']

OrderedDict([('precision@10', 0.0926),
             ('recall@10', 0.1521),
             ('map@10', 0.0736),
             ('ndcg@10', 0.1422),
             ('mrr@10', 0.2543),
             ('hit@10', 0.536),
             ('itemcoverage@10', 0.1717)])

### After training lets make final fit on trainval set

In [ ]:
config['benchmark_filename'] = ['trainval', 'val', 'test']

In [ ]:
dataset = create_dataset(config)

In [ ]:
trainval_set, valid_set, test_set = data_preparation(config, dataset)

18 Aug 11:33    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
18 Aug 11:33    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}]


In [ ]:
model = RecboleBench.initialize_with_params(
                    train_loader=trainval_set)

18 Aug 11:33    WARNING  Max value of user's history interaction records has reached 42.77929155313351% of the total.


In [ ]:
model.fit(trainval_set)

18 Aug 11:33    INFO  epoch 0 training [time: 0.01s, train loss: 254.0525]
18 Aug 11:33    INFO  epoch 1 training [time: 0.01s, train loss: 253.2876]
18 Aug 11:33    INFO  epoch 2 training [time: 0.01s, train loss: 252.1098]
18 Aug 11:33    INFO  epoch 3 training [time: 0.01s, train loss: 250.2143]
18 Aug 11:33    INFO  epoch 4 training [time: 0.01s, train loss: 247.4650]
18 Aug 11:33    INFO  epoch 5 training [time: 0.01s, train loss: 243.8431]
18 Aug 11:33    INFO  epoch 6 training [time: 0.01s, train loss: 239.6220]
18 Aug 11:33    INFO  epoch 7 training [time: 0.01s, train loss: 235.5777]
18 Aug 11:33    INFO  epoch 8 training [time: 0.01s, train loss: 232.8785]
18 Aug 11:33    INFO  epoch 9 training [time: 0.01s, train loss: 232.3856]
18 Aug 11:33    INFO  epoch 10 training [time: 0.01s, train loss: 232.9111]
18 Aug 11:33    INFO  epoch 11 training [time: 0.01s, train loss: 232.8951]
18 Aug 11:33    INFO  epoch 12 training [time: 0.01s, train loss: 232.1133]
18 Aug 11:33    INFO  

In [ ]:
ranks = model.get_relevant_ranks(test_set)
top_100_items = model.recommend_k(test_set, 100)
metrics = run_all_metrics(ranks, [5, 10, 20, 100])
coverage_metrics = []
for k in (5, 10, 20, 100):
    coverage_metrics.append(coverage(
        top_100_items,
        test_set.dataset.item_num,
        k
    ))

metrics_df = pd.DataFrame(metrics, index=[5, 10, 20, 100], columns=(
    'Precision@k', 'Recall@K', 'MAP@K', 'nDCG@k', 'MRR@k', 'HitRate@k'
))
metrics_df['Coverage@K'] = coverage_metrics

metrics_df['Time_fit'] = model.learning_time
metrics_df['Time_predict'] = model.predict_time
metrics_df

/home/recsys/miniconda/envs/rec_env/lib/python3.9/site-packages/recbole/utils/case_study.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  uid_series = torch.tensor(uid_series)


  0%|          | 0/4 [00:00<?, ?it/s]

,Precision@k,Recall@K,MAP@K,nDCG@k,MRR@k,HitRate@k,Coverage@K,Time_fit,Time_predict
5,0.157113,0.113496,0.093540,0.151451,0.261042,0.450372,0.337875,6.034429,0.048151
10,0.209672,0.197785,0.090865,0.173552,0.284857,0.630273,0.455041,6.034429,0.048151
20,0.324464,0.322734,0.104331,0.220295,0.295406,0.777916,0.604905,6.034429,0.048151
100,0.749474,0.749474,0.137642,0.351021,0.300620,0.960298,0.942779,6.034429,0.048151
